[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/semantic-chunkers/blob/main/docs/00-chunkers-intro.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/aurelio-labs/semantic-chunkers/blob/main/docs/00-chunkers-intro.ipynb)

In [1]:
!pip install -qU \
    semantic-chunkers \
    datasets==2.19.1


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


# An Intro to Semantic Chunkers

Semantic chunkers allow us to build more context aware chunks of information. We can use this for RAG, splitting video, audio, and much more.

In this example, we will stick with a simple RAG-focused example. We will learn about three different types of chunkers available to us; `StatisticalChunker`, `ConsecutiveChunker`, `CumulativeChunker`, and `RegexChunker`. To begin, we need some data.

**Note:** by using the [async methods here]([link](https://github.com/aurelio-labs/semantic-chunkers/blob/main/docs/02-chunkers-async.ipynb)) docs can be processed *40x* faster.

In [2]:
from datasets import load_dataset

data = load_dataset("jamescalam/ai-arxiv2", split="train")
data

/Users/jakit/customers/aurelio/semantic-chunkers/.venv_312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['id', 'title', 'summary', 'source', 'authors', 'categories', 'comment', 'journal_ref', 'primary_category', 'published', 'updated', 'content', 'references'],
    num_rows: 2673
})

In [3]:
content = data[3]["content"]
print(content[:1000])

# Mamba: Linear-Time Sequence Modeling with Selective State Spaces
# Albert Gu*1 and Tri Dao*2
1Machine Learning Department, Carnegie Mellon University 2Department of Computer Science, Princeton University agu@cs.cmu.edu, tri@tridao.me
# Abstract
Foundation models, now powering most of the exciting applications in deep learning, are almost universally based on the Transformer architecture and its core attention module. Many subquadratic-time architectures such as linear attention, gated convolution and recurrent models, and structured state space models (SSMs) have been developed to address Transformersâ computational ineï¬ciency on long sequences, but they have not performed as well as attention on important modalities such as language. We identify that a key weakness of such models is their inability to perform content-based reasoning, and make several improvements. First, simply letting the SSM parameters be functions of the input addresses their weakness with discrete modalities

We will keep a smaller section of content to speed up (and limit cost) for the examples.

In [4]:
content = content[:20_000]

We will experiment with different semantic chunking methods on the above text. Every chunker requires an _encoder_ for which we can choose from open source encoders via `HuggingfaceEncoder` or `FastembedEncoder`, and proprietary API encoders like `OpenAIEncoder` or `CohereEncoder`.

We will use the `OpenAIEncoder` with `text-embedding-3-small`:

In [25]:
import os
from getpass import getpass
from semantic_router.encoders import OpenAIEncoder

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "OpenAI API key: "
)

encoder = OpenAIEncoder(name="text-embedding-3-small")

## Statistical Chunking

The statistical chunking method our most robust chunking method, it uses a varying similarity threshold to identify more dynamic and local similarity splits. It offers a good balance between accuracy and efficiency _but_ can only be used for text documents (unlike the multi-modal `ConsecutiveChunker`).

The `StatisticalChunker` can automatically identify a good threshold value to use while chunking our text, so it tends to require less customization than our other chunkers.

In [26]:
from semantic_chunkers import StatisticalChunker

chunker = StatisticalChunker(encoder=encoder)

In [27]:
chunks = chunker(docs=[content])

2024-07-03 19:57:53 INFO semantic_chunkers.utils.logger Single document exceeds the maximum token limit of 300. Splitting to sentences before semantically merging.
100%|██████████| 6/6 [00:04<00:00,  1.46it/s]


Print and compare sync and async chunks.

In [29]:
chunker.print(chunks[0])

Split 1, tokens 300, triggered by: token limit
# Mamba: Linear-Time Sequence Modeling with Selective State Spaces # Albert Gu*1 and Tri Dao*2 1Machine Learning Department, Carnegie Mellon University 2Department of Computer Science, Princeton University agu@cs.cmu.edu, tri@tridao.me # Abstract Foundation models, now powering most of the exciting applications in deep learning, are almost universally based on the Transformer architecture and its core attention module. Many subquadratic-time architectures such as linear attention, gated convolution and recurrent models, and structured state space models (SSMs) have been developed to address Transformersâ computational ineï¬ ciency on long sequences, but they have not performed as well as attention on important modalities such as language. We identify that a key weakness of such models is their inability to perform content-based reasoning, and make several improvements. First, simply letting the SSM parameters be functions of the input addr

## Consecutive Chunking

Consecutive chunking is the simplest version of semantic chunking.

In [30]:
from semantic_chunkers import ConsecutiveChunker

chunker = ConsecutiveChunker(encoder=encoder, score_threshold=0.3)

In [31]:
chunks = chunker(docs=[content])

100%|██████████| 328/328 [00:00<00:00, 92287.63it/s]


In [32]:
chunker.print(chunks[0])

Split 1, tokens None, triggered by: 0.09
# Mamba:
----------------------------------------------------------------------------------------


Split 2, tokens None, triggered by: 0.10
Linear-Time Sequence Modeling with Selective State Spaces
----------------------------------------------------------------------------------------


Split 3, tokens None, triggered by: 0.25
# Albert Gu*1 and Tri Dao*2 1Machine Learning Department, Carnegie Mellon University 2Department of Computer Science, Princeton University agu@cs.cmu.edu, tri@tridao.me
----------------------------------------------------------------------------------------


Split 4, tokens None, triggered by: 0.22
# Abstract
----------------------------------------------------------------------------------------


Split 5, tokens None, triggered by: 0.30
Foundation models, now powering most of the exciting applications in deep learning, are almost universally based on the Transformer architecture and its core attention module. Many sub

## Cumulative Chunking

Cumulative chunking is a more compute intensive process, but can often provide more stable results as it is more noise resistant. However, it is _very expensive_ in both time and (if using APIs) money.

In [33]:
from semantic_chunkers import CumulativeChunker

chunker = CumulativeChunker(encoder=encoder, score_threshold=0.3)

In [34]:
chunks = chunker(docs=[content])

100%|██████████| 329/329 [03:22<00:00,  1.63it/s]


In [35]:
chunker.print(chunks[0])

Split 1, tokens None, triggered by: 0.09
# Mamba:
----------------------------------------------------------------------------------------


Split 2, tokens None, triggered by: 0.10
Linear-Time Sequence Modeling with Selective State Spaces
----------------------------------------------------------------------------------------


Split 3, tokens None, triggered by: 0.28
# Albert Gu*1 and Tri Dao*2 1Machine Learning Department, Carnegie Mellon University 2Department of Computer Science, Princeton University agu@cs.cmu.edu, tri@tridao.me
----------------------------------------------------------------------------------------


Split 4, tokens None, triggered by: 0.22
# Abstract
----------------------------------------------------------------------------------------


Split 5, tokens None, triggered by: 0.23
Foundation models, now powering most of the exciting applications in deep learning, are almost universally based on the Transformer architecture and its core attention module. Many sub

## Regex Chunking

In [6]:
from typing import List
from semantic_chunkers import RegexChunker
from semantic_chunkers.schema import Chunk

chunker = RegexChunker()
chunks: List[List[Chunk]] = chunker(docs=[content])
chunker.print(chunks[0])

Split 1, tokens 300, triggered by: token limit
# Mamba: Linear-Time Sequence Modeling with Selective State Spaces # Albert Gu*1 and Tri Dao*2 1Machine Learning Department, Carnegie Mellon University 2Department of Computer Science, Princeton University agu@cs.cmu.edu, tri@tridao.me # Abstract Foundation models, now powering most of the exciting applications in deep learning, are almost universally based on the Transformer architecture and its core attention module. Many subquadratic-time architectures such as linear attention, gated convolution and recurrent models, and structured state space models (SSMs) have been developed to address Transformersâ computational ineï¬ ciency on long sequences, but they have not performed as well as attention on important modalities such as language. We identify that a key weakness of such models is their inability to perform content-based reasoning, and make several improvements. First, simply letting the SSM parameters be functions of the input addr

---